In [1]:
import numpy as np
from random import randrange

from src.constants import p, a, b
from src.IntModP import IntModP
from src.CurvePoint import CurvePoint
from src.EllipticCurve import EllipticCurve

from src.Encoder import Encoder

In [2]:
x = IntModP(6)
y = IntModP(3)
x+=y
print(x**2)
y*=x
print(y)


81 [317]
27 [317]


In [3]:
P = CurvePoint(a, b, 8, 1)

print((-2)*P)
print(P - 3*P)


Point de E(Z/317Z) : (156 [317], 103 [317])
Point de E(Z/317Z) : (156 [317], 103 [317])


In [4]:
E = EllipticCurve(a, b)
print(E)
# E.printPoints()

y^2 = x^3 + 15 [317]x + 3 [317]


In [24]:
msg = "Message secret 123"
nMsg = Encoder.encode(msg)
print(nMsg)
print(Encoder.decode(nMsg))


6742167996113793302940144189862687263502899
Message secret 123


In [25]:
P = CurvePoint(a, b, 8, 1)
q = P.order()
E = Encoder(msg, P)
(E.dico[" "].x, E.dico[" "].y)

(None, None)

In [26]:
arr = E.stringToArray()
n = len(arr)
if n%3 != 0 :
	arr = arr + [E.dico[" "]] * (3 - n%3)
n=len(arr)
[(el.x, el.y) for el in arr]

[(127, 209),
 (166, 102),
 (120, 85),
 (120, 85),
 (8, 1),
 (19, 217),
 (166, 102),
 (None, None),
 (120, 85),
 (166, 102),
 (144, 86),
 (88, 166),
 (166, 102),
 (249, 56),
 (None, None),
 (260, 44),
 (271, 212),
 (161, 227)]

In [27]:
A = np.array([
	[-1, 5, -1], 
	[-2, 11, 7], 
	[1, -5, 2]
])

np.linalg.det(A)

-1.0

In [28]:
r = int(n/3)

M = np.array([
	[arr[j] for j in range(r)],
	[arr[j] for j in range(r, 2*r)],
	[arr[j] for j in range(2*r, n)],
])


In [29]:
Q = np.dot(A, M)

In [30]:
S = np.ndarray.flatten(Q)
np.shape(S)
# [(el.x, el.y) for el in S]

(18,)

In [31]:
# On a les points sur la courbe

# Données : 
alice = {}
public = {}
bob = {
	"msg": msg
}


In [32]:
# Alice choisit a
alice["a"] = randrange(q)

# Alice publie aP
public["aP"] = alice["a"] * P

In [33]:
# Bob choisit b
bob["b"] = randrange(q)

# Bob publie le couple (bP, M + b(aP))
public["bP"] = ( bob["b"]*P, S + bob["b"] * public["aP"] )

In [34]:
# Alice reconstitue la matrice S
c1, c2 = public["bP"]
S_out = c2 - alice["a"]*c1
# [(el.x, el.y) for el in M]

In [35]:
Q_out = np.array([
	[S_out[j] for j in range(r)],
	[S_out[j] for j in range(r, 2*r)],
	[S_out[j] for j in range(2*r, n)],
])
M_out = np.dot(
	np.int_(np.linalg.inv(A)),
	Q_out
)

In [36]:
msg_out_arr = np.ndarray.flatten(M_out)

In [37]:
final_msg = [E.dico_inv[f"{e.x},{e.y}"] for e in msg_out_arr]
"".join(final_msg)

'Message secret 123'